"""NYC Taxi Zone-Based Passenger Demand Predictor
Predicts passenger demand by bucket (single/small/medium/large) for each zone
Using both KNN and XGBoost models
"""

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import logging
from google.cloud import bigquery, storage
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
import joblib
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timedelta

# Configuration?

In [ ]:
# Configuration
PROJECT_ID = "nyctaxi-467111"
SOURCE_DATASET = "CleanSilver"
OUTPUT_DATASET = "PostMlGold"
BUCKET_NAME = "nyc_raw_data_bucket"
MODEL_FOLDER = "zone_based_models_new"

# Taxi types to process
TAXI_TYPES = ["yellow", "green"]

# Minimum trips threshold for zone modeling
MIN_TRIPS_THRESHOLD = 0

In [ ]:

# Forecast parameters
FORECAST_DAYS = 15
CONFIDENCE_LEVEL = 0.95


In [ ]:
"""# Setup logging"""

class LoggerWrapper:
    def __init__(self, name=None):
        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        self._logger = logging.getLogger(name or __name__)

    def info(self, message):
        self._logger.info(message)
        print(message)  # Simple print without [INFO] prefix

    def error(self, message, exc_info=False):
        self._logger.error(message, exc_info=exc_info)
        print(f"ERROR: {message}")
        if exc_info:
            import traceback
            traceback.print_exc()

    def warning(self, message):
        self._logger.warning(message)
        print(f"WARNING: {message}")

# Create logger instance
logger = LoggerWrapper(__name__)

"""# Initialize clients"""

bq_client = bigquery.Client(project=PROJECT_ID)
storage_client = storage.Client()

In [ ]:
# logger.addHandler(logging.StreamHandler(sys.stdout))

In [ ]:
logger.info(f"Your message here");

# Helper functions

In [ ]:
def calculate_smape(y_true, y_pred):
    """Calculate Symmetric Mean Absolute Percentage Error"""
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    mask = denominator != 0
    smape = np.zeros_like(y_true)
    smape[mask] = np.abs(y_true[mask] - y_pred[mask]) / denominator[mask]
    return 100 * np.mean(smape)


In [ ]:
def extract_features_from_bigquery(taxi_type, end_date=None):
    """Extract and prepare features from BigQuery tables."""
    logger.info(f"Extracting features for {taxi_type} taxi...")

    if end_date is None:
        # Get the latest date from available data
        latest_date_query = f"""
        SELECT MAX(DATE(pickup_datetime)) as max_date
        FROM (
            SELECT {'tpep' if taxi_type == 'yellow' else 'lpep'}_pickup_datetime as pickup_datetime
            FROM `{PROJECT_ID}.{SOURCE_DATASET}.{taxi_type}*`
        )
        """
        end_date = list(bq_client.query(latest_date_query).result())[0].max_date

    pickup_col = 'tpep_pickup_datetime' if taxi_type == 'yellow' else 'lpep_pickup_datetime'

    query = f"""
    WITH bounds AS (
        SELECT
            DATE('2024-01-01') AS min_d,
            DATE('{end_date}') AS max_d
    ),
    zones AS (
        SELECT DISTINCT PULocationID AS zone_id
        FROM `{PROJECT_ID}.{SOURCE_DATASET}.{taxi_type}*`
        WHERE PULocationID IS NOT NULL
    ),
    grid AS (
        SELECT
            d,
            hr,
            zone_id
        FROM bounds b,
        UNNEST(GENERATE_DATE_ARRAY(b.min_d, b.max_d)) AS d,
        UNNEST(GENERATE_ARRAY(0, 23)) AS hr
        CROSS JOIN zones
    ),
    raw_trips AS (
        SELECT
            DATE({pickup_col}) AS d,
            EXTRACT(HOUR FROM {pickup_col}) AS hr,
            PULocationID AS zone_id,
            CASE
                WHEN passenger_count <= 1.5 THEN 'single'
                WHEN passenger_count <= 3.5 THEN 'small'
                WHEN passenger_count <= 5.5 THEN 'medium'
                ELSE 'large'
            END AS bucket
        FROM `{PROJECT_ID}.{SOURCE_DATASET}.{taxi_type}*`
        WHERE DATE({pickup_col}) >= '2024-01-01'
          AND DATE({pickup_col}) <= '{end_date}'
          AND PULocationID IS NOT NULL
    ),
    trip_counts AS (
        SELECT
            d, hr, zone_id,
            COUNTIF(bucket = 'single') AS y_single,
            COUNTIF(bucket = 'small') AS y_small,
            COUNTIF(bucket = 'medium') AS y_medium,
            COUNTIF(bucket = 'large') AS y_large
        FROM raw_trips
        GROUP BY d, hr, zone_id
    ),
    filled AS (
        SELECT
            g.d, g.hr, g.zone_id,
            COALESCE(c.y_single, 0) AS y_single,
            COALESCE(c.y_small, 0) AS y_small,
            COALESCE(c.y_medium, 0) AS y_medium,
            COALESCE(c.y_large, 0) AS y_large,
            COALESCE(c.y_single + c.y_small + c.y_medium + c.y_large, 0) AS total_trips
        FROM grid g
        LEFT JOIN trip_counts c USING (d, hr, zone_id)
    ),
    with_features AS (
        SELECT
            f.*,
            EXTRACT(DAYOFWEEK FROM f.d) AS dow,
            EXTRACT(MONTH FROM f.d) AS month,
            -- Lag features
            LAG(y_single, 1) OVER (PARTITION BY zone_id, hr ORDER BY d) AS y_single_lag1,
            LAG(y_single, 7) OVER (PARTITION BY zone_id, hr ORDER BY d) AS y_single_lag7,
            LAG(y_small, 1) OVER (PARTITION BY zone_id, hr ORDER BY d) AS y_small_lag1,
            LAG(y_small, 7) OVER (PARTITION BY zone_id, hr ORDER BY d) AS y_small_lag7,
            LAG(y_medium, 1) OVER (PARTITION BY zone_id, hr ORDER BY d) AS y_medium_lag1,
            LAG(y_medium, 7) OVER (PARTITION BY zone_id, hr ORDER BY d) AS y_medium_lag7,
            LAG(y_large, 1) OVER (PARTITION BY zone_id, hr ORDER BY d) AS y_large_lag1,
            LAG(y_large, 7) OVER (PARTITION BY zone_id, hr ORDER BY d) AS y_large_lag7,
            LAG(total_trips, 1) OVER (PARTITION BY zone_id, hr ORDER BY d) AS total_trips_lag1,
            LAG(total_trips, 7) OVER (PARTITION BY zone_id, hr ORDER BY d) AS total_trips_lag7,
            -- Rolling averages
            AVG(total_trips) OVER (
                PARTITION BY zone_id, hr
                ORDER BY d
                ROWS BETWEEN 7 PRECEDING AND 1 PRECEDING
            ) AS total_trips_7dma,
            AVG(total_trips) OVER (
                PARTITION BY zone_id, hr
                ORDER BY d
                ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING
            ) AS total_trips_3dma,
            -- Zone-level statistics
            AVG(total_trips) OVER (PARTITION BY zone_id) AS zone_avg_trips,
            STDDEV(total_trips) OVER (PARTITION BY zone_id) AS zone_std_trips
        FROM filled f
    )
    SELECT * FROM with_features
    WHERE d >= DATE('2024-01-08')  -- Ensure we have lag features
    ORDER BY zone_id, d, hr
    """

    df = bq_client.query(query).to_dataframe()
    logger.info(f"Extracted {len(df):,} rows of feature data")
    print(f"Extracted {len(df):,} rows of feature data")
    return df

In [ ]:
def prepare_features(df):
    """Add cyclical time features and handle missing values."""
    # Fill NaN values
    lag_cols = [c for c in df.columns if '_lag' in c]
    df[lag_cols] = df[lag_cols].fillna(0.0)
    df['total_trips_7dma'] = df['total_trips_7dma'].fillna(0.0)
    df['total_trips_3dma'] = df['total_trips_3dma'].fillna(0.0)
    df['zone_avg_trips'] = df['zone_avg_trips'].fillna(0.0)
    df['zone_std_trips'] = df['zone_std_trips'].fillna(1.0)

    # Convert numeric columns to float
    numeric_cols = ['total_trips', 'y_single', 'y_small', 'y_medium', 'y_large',
                   'zone_avg_trips', 'zone_std_trips', 'total_trips_3dma', 'total_trips_7dma'] + lag_cols
    for col in numeric_cols:
        if col in df.columns:
            df[col] = df[col].astype('float64')

    # Add cyclical time features
    df['hour_sin'] = np.sin(2 * np.pi * df['hr'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hr'] / 24)
    df['dow_sin'] = np.sin(2 * np.pi * df['dow'] / 7)
    df['dow_cos'] = np.cos(2 * np.pi * df['dow'] / 7)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

    return df

In [ ]:
def build_preprocessor_for_zone():
    """Build feature preprocessor for zone-specific models."""
    # No categorical features since we're modeling per zone
    num_features = [
        'hr', 'dow', 'month',
        'total_trips', 'total_trips_3dma', 'total_trips_7dma',
        'zone_avg_trips', 'zone_std_trips',
        # All lag features
        'y_single_lag1', 'y_single_lag7',
        'y_small_lag1', 'y_small_lag7',
        'y_medium_lag1', 'y_medium_lag7',
        'y_large_lag1', 'y_large_lag7',
        'total_trips_lag1', 'total_trips_lag7',
        # Cyclical features
        'hour_sin', 'hour_cos',
        'dow_sin', 'dow_cos',
        'month_sin', 'month_cos'
    ]

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), num_features)
        ]
    )

    return preprocessor, num_features


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.gridspec import GridSpec

def train_zone_models(zone_data, zone_id, test_days=7):
    """Train both KNN and XGBoost models for a specific zone."""
    logger.info(f"Starting training for zone {zone_id}")

    # Split data
    zone_data['d'] = pd.to_datetime(zone_data['d'])
    cutoff_date = zone_data['d'].max() - timedelta(days=test_days)

    logger.info(f"Zone {zone_id}: Data range from {zone_data['d'].min()} to {zone_data['d'].max()}")
    logger.info(f"Zone {zone_id}: Using cutoff date {cutoff_date} (last {test_days} days for testing)")

    train_df = zone_data[zone_data['d'] <= cutoff_date].copy()
    test_df = zone_data[zone_data['d'] > cutoff_date].copy()

    logger.info(f"Zone {zone_id}: Train samples: {len(train_df)}, Test samples: {len(test_df)}")

    # Check if zone has enough data
    if len(train_df) < MIN_TRIPS_THRESHOLD:
        logger.info(f"Zone {zone_id}: Insufficient data ({len(train_df)} < {MIN_TRIPS_THRESHOLD}), skipping")
        return None, None, None, None, None, None

    # Prepare features
    logger.info(f"Zone {zone_id}: Building preprocessor and extracting features")
    preprocessor, feature_cols = build_preprocessor_for_zone()
    X_train = preprocessor.fit_transform(train_df[feature_cols])
    X_test = preprocessor.transform(test_df[feature_cols])

    logger.info(f"Zone {zone_id}: Feature dimensions - Train: {X_train.shape}, Test: {X_test.shape}")

    # Target columns
    targets = ['y_single', 'y_small', 'y_medium', 'y_large']
    y_train = train_df[targets].values
    y_test = test_df[targets].values

    logger.info(f"Zone {zone_id}: Target shape - Train: {y_train.shape}, Test: {y_test.shape}")
    logger.info(f"Zone {zone_id}: Average trips per bucket - Single: {y_train[:, 0].mean():.2f}, Small: {y_train[:, 1].mean():.2f}, Medium: {y_train[:, 2].mean():.2f}, Large: {y_train[:, 3].mean():.2f}")

    # Train XGBoost model
    logger.info(f"Zone {zone_id}: Training XGBoost model")
    xgb_model = MultiOutputRegressor(
        XGBRegressor(
            objective='count:poisson',
            max_delta_step=1,
            n_estimators=100,
            learning_rate=0.05,
            max_depth=4,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
            n_jobs=1
        )
    )
    xgb_model.fit(X_train, y_train)
    xgb_pred = xgb_model.predict(X_test)
    logger.info(f"Zone {zone_id}: XGBoost training completed")

    # Train KNN model
    n_neighbors = min(20, len(X_train) // 10)
    logger.info(f"Zone {zone_id}: Training KNN model with {n_neighbors} neighbors")
    knn_model = MultiOutputRegressor(
        KNeighborsRegressor(
            n_neighbors=n_neighbors,
            weights='distance',
            metric='manhattan'
        )
    )
    knn_model.fit(X_train, y_train)
    knn_pred = knn_model.predict(X_test)
    logger.info(f"Zone {zone_id}: KNN training completed")

    # Calculate residuals for confidence intervals
    logger.info(f"Zone {zone_id}: Calculating residuals for confidence intervals")
    xgb_residuals = y_train - xgb_model.predict(X_train)
    knn_residuals = y_train - knn_model.predict(X_train)

    xgb_residual_std = np.std(xgb_residuals, axis=0)
    knn_residual_std = np.std(knn_residuals, axis=0)

    logger.info(f"Zone {zone_id}: XGB residual std: {xgb_residual_std}")
    logger.info(f"Zone {zone_id}: KNN residual std: {knn_residual_std}")

    # Calculate metrics
    logger.info(f"Zone {zone_id}: Calculating performance metrics")
    metrics = {
        'xgb': {},
        'knn': {}
    }

    test_predictions = test_df[['d', 'hr', 'dow', 'month']].copy()
    test_predictions['zone_id'] = zone_id
    test_predictions['prediction_type'] = 'test'  # Mark as test data

    for i, target in enumerate(targets):
        test_predictions[f'actual_{target}'] = y_test[:, i]
        test_predictions[f'xgb_pred_{target}'] = xgb_pred[:, i]
        test_predictions[f'knn_pred_{target}'] = knn_pred[:, i]
        # XGBoost metrics
        xgb_mae = mean_absolute_error(y_test[:, i], xgb_pred[:, i])
        xgb_rmse = np.sqrt(mean_squared_error(y_test[:, i], xgb_pred[:, i]))
        xgb_smape = calculate_smape(y_test[:, i], xgb_pred[:, i])

        metrics['xgb'][target] = {
            'mae': xgb_mae,
            'rmse': xgb_rmse,
            'smape': xgb_smape
        }

        # KNN metrics
        knn_mae = mean_absolute_error(y_test[:, i], knn_pred[:, i])
        knn_rmse = np.sqrt(mean_squared_error(y_test[:, i], knn_pred[:, i]))
        knn_smape = calculate_smape(y_test[:, i], knn_pred[:, i])

        metrics['knn'][target] = {
            'mae': knn_mae,
            'rmse': knn_rmse,
            'smape': knn_smape
        }

        logger.info(f"Zone {zone_id}: {target} - XGB (MAE: {xgb_mae:.2f}, RMSE: {xgb_rmse:.2f}, SMAPE: {xgb_smape:.2f}%) | KNN (MAE: {knn_mae:.2f}, RMSE: {knn_rmse:.2f}, SMAPE: {knn_smape:.2f}%)")

    # Create visualization
    logger.info(f"Zone {zone_id}: Creating performance visualization")
    # create_zone_performance_plot(zone_id, test_df, y_test, xgb_pred, knn_pred, metrics, targets)

    logger.info(f"Zone {zone_id}: Training completed successfully")
    return xgb_model, knn_model, preprocessor, metrics, feature_cols, (xgb_residual_std, knn_residual_std), test_predictions


def create_zone_performance_plot(zone_id, test_df, y_test, xgb_pred, knn_pred, metrics, targets):
    """Create a comprehensive performance plot for a zone."""
    # Create figure with subplots
    fig = plt.figure(figsize=(20, 12))
    gs = GridSpec(3, 2, figure=fig, hspace=0.3, wspace=0.3)

    # Main title
    fig.suptitle(f'Zone {zone_id} - Model Performance Analysis', fontsize=16, fontweight='bold')

    # Colors for each passenger bucket
    colors = {'y_single': '#1f77b4', 'y_small': '#ff7f0e', 'y_medium': '#2ca02c', 'y_large': '#d62728'}

    # Create datetime index for x-axis
    test_df['datetime'] = pd.to_datetime(test_df['d']) + pd.to_timedelta(test_df['hr'], unit='h')

    # Plot 1: Time series comparison for all buckets
    ax1 = fig.add_subplot(gs[0, :])

    # Plot actual vs predicted for total trips
    total_actual = y_test.sum(axis=1)
    total_xgb = xgb_pred.sum(axis=1)
    total_knn = knn_pred.sum(axis=1)

    ax1.plot(test_df['datetime'], total_actual, 'k-', label='Actual', linewidth=2)
    ax1.plot(test_df['datetime'], total_xgb, 'b--', label='XGBoost', linewidth=1.5, alpha=0.7)
    ax1.plot(test_df['datetime'], total_knn, 'r:', label='KNN', linewidth=1.5, alpha=0.7)

    ax1.set_title('Total Trips: Actual vs Predicted', fontsize=12)
    ax1.set_xlabel('Date/Time')
    ax1.set_ylabel('Total Trips')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d %H:00'))
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    # Add metrics annotation
    total_xgb_mae = mean_absolute_error(total_actual, total_xgb)
    total_knn_mae = mean_absolute_error(total_actual, total_knn)
    ax1.text(0.02, 0.95, f'XGB MAE: {total_xgb_mae:.2f}\nKNN MAE: {total_knn_mae:.2f}',
             transform=ax1.transAxes, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

    # Plots 2-5: Individual bucket performance
    for idx, (i, target) in enumerate(zip(range(4), targets)):
        row = (idx // 2) + 1
        col = idx % 2
        ax = fig.add_subplot(gs[row, col])

        actual = y_test[:, i]
        xgb = xgb_pred[:, i]
        knn = knn_pred[:, i]

        # Plot time series
        ax.plot(test_df['datetime'], actual, 'k-', label='Actual', linewidth=1.5)
        ax.plot(test_df['datetime'], xgb, 'b--', label='XGBoost', linewidth=1, alpha=0.7)
        ax.plot(test_df['datetime'], knn, 'r:', label='KNN', linewidth=1, alpha=0.7)

        ax.set_title(f'{target.replace("y_", "").capitalize()} Passengers', fontsize=10)
        ax.set_xlabel('Date/Time', fontsize=8)
        ax.set_ylabel('Trip Count', fontsize=8)
        ax.legend(fontsize=8)
        ax.grid(True, alpha=0.3)
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
        plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

        # Add metrics box
        xgb_metrics = metrics['xgb'][target]
        knn_metrics = metrics['knn'][target]

        metrics_text = (f"XGBoost:\n"
                       f"  MAE: {xgb_metrics['mae']:.2f}\n"
                       f"  RMSE: {xgb_metrics['rmse']:.2f}\n"
                       f"  SMAPE: {xgb_metrics['smape']:.1f}%\n"
                       f"KNN:\n"
                       f"  MAE: {knn_metrics['mae']:.2f}\n"
                       f"  RMSE: {knn_metrics['rmse']:.2f}\n"
                       f"  SMAPE: {knn_metrics['smape']:.1f}%")

        ax.text(0.02, 0.95, metrics_text, transform=ax.transAxes,
                verticalalignment='top', fontsize=7,
                bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.7))

    # Save plot
    plt.tight_layout()

    # Create directory if it doesn't exist
    import os
    plot_dir = f"zone_performance_plots/{zone_id}"
    os.makedirs(plot_dir, exist_ok=True)

    # Save with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    plot_path = f"{plot_dir}/performance_{timestamp}.png"
    plt.savefig(plot_path, dpi=150, bbox_inches='tight')
    plt.close()

    logger.info(f"Zone {zone_id}: Performance plot saved to {plot_path}")

    # Also save to GCS if needed
    try:
        bucket = storage_client.get_bucket(BUCKET_NAME)
        blob = bucket.blob(f"zone_performance_plots/zone_{zone_id}_performance_{timestamp}.png")
        blob.upload_from_filename(plot_path)
        logger.info(f"Zone {zone_id}: Performance plot uploaded to GCS")
    except Exception as e:
        logger.warning(f"Zone {zone_id}: Failed to upload plot to GCS: {str(e)}")


def create_summary_performance_plot(all_metrics, taxi_type):
    """Create a summary plot showing performance across all zones."""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'{taxi_type.capitalize()} Taxi - Model Performance Summary Across All Zones', fontsize=14)

    targets = ['y_single', 'y_small', 'y_medium', 'y_large']

    for idx, target in enumerate(targets):
        ax = axes[idx // 2, idx % 2]

        # Collect metrics for all zones
        zones = []
        xgb_maes = []
        knn_maes = []
        xgb_smapes = []
        knn_smapes = []

        for zone_id, zone_metrics in all_metrics.items():
            zones.append(zone_id)
            xgb_maes.append(zone_metrics['xgb'][target]['mae'])
            knn_maes.append(zone_metrics['knn'][target]['mae'])
            xgb_smapes.append(zone_metrics['xgb'][target]['smape'])
            knn_smapes.append(zone_metrics['knn'][target]['smape'])

        # Create bar plot
        x = np.arange(len(zones))
        width = 0.35

        ax.bar(x - width/2, xgb_maes, width, label='XGBoost MAE', alpha=0.7)
        ax.bar(x + width/2, knn_maes, width, label='KNN MAE', alpha=0.7)

        ax.set_title(f'{target.replace("y_", "").capitalize()} Passengers - MAE by Zone')
        ax.set_xlabel('Zone ID')
        ax.set_ylabel('MAE')
        ax.legend()

        # Limit x-axis labels for readability
        if len(zones) > 20:
            step = len(zones) // 20
            ax.set_xticks(x[::step])
            ax.set_xticklabels(zones[::step], rotation=45)
        else:
            ax.set_xticks(x)
            ax.set_xticklabels(zones, rotation=45)

    plt.tight_layout()

    # Save summary plot
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    summary_path = f"zone_performance_plots/{taxi_type}_summary_{timestamp}.png"
    plt.savefig(summary_path, dpi=150, bbox_inches='tight')
    plt.close()

    logger.info(f"Summary performance plot saved to {summary_path}")

In [ ]:
def predict_future_iterative(zone_data, zone_models, zone_id, last_date, residual_stds, days_ahead=15):
    """Generate iterative predictions for a specific zone."""
    logger.info(f"Zone {zone_id}: Starting iterative predictions for {days_ahead} days ahead")

    if zone_id not in zone_models:
        logger.warning(f"Zone {zone_id}: No trained models found, skipping predictions")
        return None

    models = zone_models[zone_id]
    xgb_residual_std, knn_residual_std = residual_stds[zone_id]

    logger.info(f"Zone {zone_id}: Residual STDs - XGB: {xgb_residual_std}, KNN: {knn_residual_std}")

    # Calculate z-score for confidence intervals
    z_score = 1.96 if CONFIDENCE_LEVEL == 0.95 else 2.576
    logger.info(f"Zone {zone_id}: Using confidence level {CONFIDENCE_LEVEL} (z-score: {z_score})")

    # Prepare recent history for lag features
    zone_data = zone_data.sort_values(['d', 'hr'])
    recent_history = {
        'y_single': list(zone_data['y_single'].iloc[-168:].values),
        'y_small': list(zone_data['y_small'].iloc[-168:].values),
        'y_medium': list(zone_data['y_medium'].iloc[-168:].values),
        'y_large': list(zone_data['y_large'].iloc[-168:].values),
        'total_trips': list(zone_data['total_trips'].iloc[-168:].values)
    }

    logger.info(f"Zone {zone_id}: Prepared recent history with {len(recent_history['total_trips'])} hours of data")
    logger.info(f"Zone {zone_id}: Last historical values - Single: {recent_history['y_single'][-1]:.2f}, Small: {recent_history['y_small'][-1]:.2f}, Medium: {recent_history['y_medium'][-1]:.2f}, Large: {recent_history['y_large'][-1]:.2f}")

    # Store predictions
    future_predictions = []
    xgb_predictions = {'y_single': [], 'y_small': [], 'y_medium': [], 'y_large': []}
    knn_predictions = {'y_single': [], 'y_small': [], 'y_medium': [], 'y_large': []}

    # Generate predictions iteratively
    future_hours = days_ahead * 24

    # Handle the date/datetime conversion more simply
    # Create a datetime object for the start of predictions
    base_date = pd.to_datetime(last_date)
    start_datetime = base_date + timedelta(days=1)

    logger.info(f"Zone {zone_id}: Generating {future_hours} hourly predictions starting from {start_datetime.date()}")

   # Log progress every 24 hours
    for h in range(future_hours):
        pred_time = start_datetime + timedelta(hours=h)

        if h % 24 == 0:
            logger.info(f"Zone {zone_id}: Predicting day {h//24 + 1}/{days_ahead} - {pred_time.date()}")

        # Create feature row
        future_features = {}

        # Time features
        future_features['hr'] = pred_time.hour
        future_features['dow'] = pred_time.weekday() + 1  # Match BigQuery's DAYOFWEEK
        future_features['month'] = pred_time.month

        # Cyclical features
        future_features['hour_sin'] = np.sin(2 * np.pi * pred_time.hour / 24)
        future_features['hour_cos'] = np.cos(2 * np.pi * pred_time.hour / 24)
        future_features['dow_sin'] = np.sin(2 * np.pi * (pred_time.weekday() + 1) / 7)
        future_features['dow_cos'] = np.cos(2 * np.pi * (pred_time.weekday() + 1) / 7)
        future_features['month_sin'] = np.sin(2 * np.pi * pred_time.month / 12)
        future_features['month_cos'] = np.cos(2 * np.pi * pred_time.month / 12)

        # Zone statistics (use historical values)
        future_features['zone_avg_trips'] = zone_data['zone_avg_trips'].iloc[-1]
        future_features['zone_std_trips'] = zone_data['zone_std_trips'].iloc[-1]

        # Lag features - use recent history or predictions
        for bucket in ['y_single', 'y_small', 'y_medium', 'y_large']:
            if h == 0:
                # First prediction - use historical data
                future_features[f'{bucket}_lag1'] = recent_history[bucket][-1]
                future_features[f'{bucket}_lag7'] = recent_history[bucket][-24*7] if len(recent_history[bucket]) >= 24*7 else recent_history[bucket][0]
            else:
                # Use previous predictions
                future_features[f'{bucket}_lag1'] = xgb_predictions[bucket][-1]  # Use XGB predictions for consistency
                if h >= 24*7:
                    future_features[f'{bucket}_lag7'] = xgb_predictions[bucket][-24*7]
                else:
                    idx = -(24*7 - h)
                    future_features[f'{bucket}_lag7'] = recent_history[bucket][idx] if abs(idx) < len(recent_history[bucket]) else recent_history[bucket][0]

        # Total trips lag features
        if h == 0:
            future_features['total_trips_lag1'] = recent_history['total_trips'][-1]
            future_features['total_trips_lag7'] = recent_history['total_trips'][-24*7] if len(recent_history['total_trips']) >= 24*7 else recent_history['total_trips'][0]
        else:
            # Sum of all bucket predictions
            future_features['total_trips_lag1'] = sum(xgb_predictions[b][-1] for b in ['y_single', 'y_small', 'y_medium', 'y_large'])
            if h >= 24*7:
                future_features['total_trips_lag7'] = sum(xgb_predictions[b][-24*7] for b in ['y_single', 'y_small', 'y_medium', 'y_large'])
            else:
                idx = -(24*7 - h)
                future_features['total_trips_lag7'] = recent_history['total_trips'][idx] if abs(idx) < len(recent_history['total_trips']) else recent_history['total_trips'][0]

        # Calculate rolling features
        recent_total = recent_history['total_trips'] + [sum(xgb_predictions[b][i] for b in ['y_single', 'y_small', 'y_medium', 'y_large']) for i in range(len(xgb_predictions['y_single']))]

        future_features['total_trips_3dma'] = np.mean(recent_total[-3*24:]) if len(recent_total) >= 3*24 else np.mean(recent_history['total_trips'][-3*24:])
        future_features['total_trips_7dma'] = np.mean(recent_total[-7*24:]) if len(recent_total) >= 7*24 else np.mean(recent_history['total_trips'][-7*24:])

        # Current total trips (use historical average for same hour/dow)
        hist_same_time = zone_data[(zone_data['hr'] == pred_time.hour) & (zone_data['dow'] == pred_time.weekday() + 1)]
        future_features['total_trips'] = hist_same_time['total_trips'].mean() if len(hist_same_time) > 0 else zone_data['total_trips'].mean()

        # Create DataFrame for prediction
        future_df = pd.DataFrame([future_features])

        # Make predictions
        X_future = models['preprocessor'].transform(future_df[models['feature_cols']])

        # XGBoost predictions
        xgb_pred = models['xgb'].predict(X_future)[0]
        xgb_pred = np.maximum(0, xgb_pred)  # Ensure non-negative

        # KNN predictions
        knn_pred = models['knn'].predict(X_future)[0]
        knn_pred = np.maximum(0, knn_pred)  # Ensure non-negative

        # Log predictions for key hours
        # if h % 24 == 0 or h == 0:
        #     logger.info(f"Zone {zone_id}: Hour {h} predictions - XGB: {xgb_pred}, KNN: {knn_pred}")

        # Store predictions
        for i, bucket in enumerate(['y_single', 'y_small', 'y_medium', 'y_large']):
            xgb_predictions[bucket].append(xgb_pred[i])
            knn_predictions[bucket].append(knn_pred[i])

            # Update recent history
            recent_history[bucket].append(xgb_pred[i])
            if len(recent_history[bucket]) > 168:
                recent_history[bucket].pop(0)

        # Update total trips history
        recent_history['total_trips'].append(sum(xgb_pred))
        if len(recent_history['total_trips']) > 168:
            recent_history['total_trips'].pop(0)

        # Create result row
        result = {
            'd': pred_time.date(),
            'hr': pred_time.hour,
            'zone_id': zone_id,
            'dow': pred_time.weekday() + 1,
            'month': pred_time.month,
            'prediction_type': 'future'
        }

        # Add predictions with confidence intervals
        uncertainty_factor = 1 + 0.02 * h  # 2% increase per hour for future uncertainty

        for i, bucket in enumerate(['y_single', 'y_small', 'y_medium', 'y_large']):
            # XGBoost predictions with CI
            result[f'xgb_pred_{bucket}'] = xgb_pred[i]
            result[f'xgb_pred_{bucket}_lower'] = max(0, xgb_pred[i] - z_score * xgb_residual_std[i] * uncertainty_factor)
            result[f'xgb_pred_{bucket}_upper'] = xgb_pred[i] + z_score * xgb_residual_std[i] * uncertainty_factor

            # KNN predictions with CI
            result[f'knn_pred_{bucket}'] = knn_pred[i]
            result[f'knn_pred_{bucket}_lower'] = max(0, knn_pred[i] - z_score * knn_residual_std[i] * uncertainty_factor)
            result[f'knn_pred_{bucket}_upper'] = knn_pred[i] + z_score * knn_residual_std[i] * uncertainty_factor

        future_predictions.append(result)

    # Summary statistics
    predictions_df = pd.DataFrame(future_predictions)

    logger.info(f"Zone {zone_id}: Prediction complete. Generated {len(predictions_df)} hourly predictions")
    logger.info(f"Zone {zone_id}: Average daily totals - XGB: {predictions_df[[f'xgb_pred_{b}' for b in ['y_single', 'y_small', 'y_medium', 'y_large']]].sum(axis=1).groupby(predictions_df['d']).sum().mean():.2f}")
    logger.info(f"Zone {zone_id}: Average daily totals - KNN: {predictions_df[[f'knn_pred_{b}' for b in ['y_single', 'y_small', 'y_medium', 'y_large']]].sum(axis=1).groupby(predictions_df['d']).sum().mean():.2f}")

    # Log prediction ranges
    for bucket in ['y_single', 'y_small', 'y_medium', 'y_large']:
        xgb_min = predictions_df[f'xgb_pred_{bucket}'].min()
        xgb_max = predictions_df[f'xgb_pred_{bucket}'].max()
        knn_min = predictions_df[f'knn_pred_{bucket}'].min()
        knn_max = predictions_df[f'knn_pred_{bucket}'].max()
        logger.info(f"Zone {zone_id}: {bucket} prediction ranges - XGB: [{xgb_min:.2f}, {xgb_max:.2f}], KNN: [{knn_min:.2f}, {knn_max:.2f}]")

    return predictions_df

In [ ]:
def train_all_zones(df, taxi_type, max_workers=4):
    """Train models for all zones in parallel."""
    zones = df['zone_id'].unique()
    zone_models = {}
    all_metrics = {}
    residual_stds = {}
    test_predictions = {}

    logger.info(f"="*80)
    logger.info(f"Starting parallel training for {taxi_type} taxi")
    logger.info(f"Total zones to process: {len(zones)}")
    logger.info(f"Max parallel workers: {max_workers}")
    logger.info(f"Data date range: {df['d'].min()} to {df['d'].max()}")
    logger.info(f"="*80)

    # Get zone statistics
    zone_stats = df.groupby('zone_id')['total_trips'].agg(['count', 'sum', 'mean']).reset_index()
    logger.info(f"Zone statistics:")
    logger.info(f"  - Zones with > 1000 samples: {len(zone_stats[zone_stats['count'] > 1000])}")
    logger.info(f"  - Zones with > 10000 samples: {len(zone_stats[zone_stats['count'] > 10000])}")
    logger.info(f"  - Total trips across all zones: {zone_stats['sum'].sum():,.0f}")

    # Track processing stats
    successful_zones = 0
    skipped_zones = 0
    failed_zones = 0
    start_time = datetime.now()

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        logger.info(f"Submitting {len(zones)} zone training tasks to thread pool...")

        future_to_zone = {
            executor.submit(
                train_zone_models,
                df[df['zone_id'] == zone].copy(),
                zone
            ): zone
            for zone in zones
        }

        logger.info(f"All tasks submitted. Processing results...")

        for idx, future in enumerate(tqdm(as_completed(future_to_zone), total=len(zones))):
            zone = future_to_zone[future]

            # Log progress every 10 zones
            if (idx + 1) % 10 == 0:
                elapsed = (datetime.now() - start_time).total_seconds()
                rate = (idx + 1) / elapsed if elapsed > 0 else 0
                eta = (len(zones) - idx - 1) / rate if rate > 0 else 0
                logger.info(f"Progress: {idx + 1}/{len(zones)} zones completed ({(idx + 1)/len(zones)*100:.1f}%) - Rate: {rate:.2f} zones/sec - ETA: {eta/60:.1f} min")

            try:
                result = future.result()
                if result[0] is not None:
                    xgb_model, knn_model, preprocessor, metrics, feature_cols, residual_std, test_preds  = result
                    zone_models[zone] = {
                        'xgb': xgb_model,
                        'knn': knn_model,
                        'preprocessor': preprocessor,
                        'feature_cols': feature_cols
                    }
                    all_metrics[zone] = metrics
                    residual_stds[zone] = residual_std
                    test_predictions[zone] = test_preds
                    successful_zones += 1

                    # Log best performing model for this zone
                    avg_xgb_smape = np.mean([metrics['xgb'][target]['smape'] for target in metrics['xgb']])
                    avg_knn_smape = np.mean([metrics['knn'][target]['smape'] for target in metrics['knn']])
                    best_model = 'XGB' if avg_xgb_smape < avg_knn_smape else 'KNN'

                    logger.info(f"Zone {zone} trained successfully - Best model: {best_model} (XGB SMAPE: {avg_xgb_smape:.1f}%, KNN SMAPE: {avg_knn_smape:.1f}%)")
                else:
                    skipped_zones += 1
                    logger.warning(f"Zone {zone} skipped - insufficient data")

            except Exception as e:
                failed_zones += 1
                logger.error(f"Error training zone {zone}: {str(e)}")
                logger.error(f"Zone {zone} stack trace:", exc_info=True)

    # Final summary
    total_time = (datetime.now() - start_time).total_seconds()
    logger.info(f"="*80)
    logger.info(f"Training completed for {taxi_type} taxi")
    logger.info(f"Total time: {total_time/60:.2f} minutes ({total_time:.1f} seconds)")
    logger.info(f"Results:")
    logger.info(f"  - Successful zones: {successful_zones}/{len(zones)} ({successful_zones/len(zones)*100:.1f}%)")
    logger.info(f"  - Skipped zones: {skipped_zones}/{len(zones)} ({skipped_zones/len(zones)*100:.1f}%)")
    logger.info(f"  - Failed zones: {failed_zones}/{len(zones)} ({failed_zones/len(zones)*100:.1f}%)")
    logger.info(f"  - Average time per zone: {total_time/len(zones):.2f} seconds")

    # Performance summary across all zones
    if all_metrics:
        logger.info(f"\nPerformance summary across all trained zones:")
        for target in ['y_single', 'y_small', 'y_medium', 'y_large']:
            xgb_smapes = [metrics['xgb'][target]['smape'] for metrics in all_metrics.values()]
            knn_smapes = [metrics['knn'][target]['smape'] for metrics in all_metrics.values()]

            logger.info(f"  {target}:")
            logger.info(f"    - XGB: Mean SMAPE={np.mean(xgb_smapes):.1f}%, Median={np.median(xgb_smapes):.1f}%, Std={np.std(xgb_smapes):.1f}%")
            logger.info(f"    - KNN: Mean SMAPE={np.mean(knn_smapes):.1f}%, Median={np.median(knn_smapes):.1f}%, Std={np.std(knn_smapes):.1f}%")

    # Identify best and worst performing zones
    if all_metrics:
        zone_avg_smapes = {}
        for zone, metrics in all_metrics.items():
            avg_smape = np.mean([metrics['xgb'][target]['smape'] for target in metrics['xgb']])
            zone_avg_smapes[zone] = avg_smape

        sorted_zones = sorted(zone_avg_smapes.items(), key=lambda x: x[1])
        logger.info(f"\nTop 5 best performing zones (lowest SMAPE):")
        for zone, smape in sorted_zones[:5]:
            logger.info(f"  - Zone {zone}: {smape:.1f}%")

        logger.info(f"\nTop 5 worst performing zones (highest SMAPE):")
        for zone, smape in sorted_zones[-5:]:
            logger.info(f"  - Zone {zone}: {smape:.1f}%")

    logger.info(f"="*80)

    return zone_models, all_metrics, residual_stds, df['d'].max(), test_predictions

In [ ]:
def save_results(all_predictions, all_metrics, test_predictions, taxi_type):
    """Save predictions and metrics to BigQuery."""
    logger.info(f"="*80)
    logger.info(f"Starting to save results for {taxi_type} taxi")
    logger.info(f"="*80)

    # Combine test and future predictions
    all_combined_predictions = []

    # Add test predictions
    for zone_id, test_preds in test_predictions.items():
        all_combined_predictions.append(test_preds)

    # Add future predictions
    all_combined_predictions.extend(all_predictions)

    # Combine all predictions (ONLY ONCE)
    logger.info(f"Combining test predictions from {len(test_predictions)} zones and future predictions from {len(all_predictions)} zones")
    predictions_df = pd.concat(all_combined_predictions, ignore_index=True)

    if 'd' in predictions_df.columns:
        predictions_df['d'] = pd.to_datetime(predictions_df['d']).dt.date

    logger.info(f"Total prediction rows: {len(predictions_df):,}")
    logger.info(f"  - Test predictions: {len(predictions_df[predictions_df['prediction_type'] == 'test']):,}")
    logger.info(f"  - Future predictions: {len(predictions_df[predictions_df['prediction_type'] == 'future']):,}")
    logger.info(f"Date range: {predictions_df['d'].min()} to {predictions_df['d'].max()}")
    logger.info(f"Unique zones in predictions: {predictions_df['zone_id'].nunique()}")

    # Prepare output tables
    xgb_table = f"{PROJECT_ID}.{OUTPUT_DATASET}.fleet_recommender_{taxi_type}_predictions_new_xgb"
    knn_table = f"{PROJECT_ID}.{OUTPUT_DATASET}.fleet_recommender_{taxi_type}_predictions_new_knn"
    metrics_table = f"{PROJECT_ID}.{OUTPUT_DATASET}.fleet_recommender_{taxi_type}_metrics_new"

    logger.info(f"Output tables:")
    logger.info(f"  - XGBoost predictions: {xgb_table}")
    logger.info(f"  - KNN predictions: {knn_table}")
    logger.info(f"  - Metrics: {metrics_table}")

    # Add metadata
    predictions_df['taxi_type'] = taxi_type
    predictions_df['prediction_timestamp'] = datetime.now()
    predictions_df['date'] = predictions_df['d']

    logger.info(f"Added metadata - prediction timestamp: {predictions_df['prediction_timestamp'].iloc[0]}")

    # Initialize column lists
    xgb_cols = ['taxi_type', 'date', 'hr', 'zone_id', 'dow', 'month', 'prediction_timestamp', 'prediction_type']
    knn_cols = ['taxi_type', 'date', 'hr', 'zone_id', 'dow', 'month', 'prediction_timestamp', 'prediction_type']

    # Add columns for each bucket
    for bucket in ['y_single', 'y_small', 'y_medium', 'y_large']:
        # Add actual values if they exist
        if f'actual_{bucket}' in predictions_df.columns:
            xgb_cols.append(f'actual_{bucket}')
            knn_cols.append(f'actual_{bucket}')

        # Add XGB predictions
        xgb_cols.extend([f'xgb_pred_{bucket}', f'xgb_pred_{bucket}_lower', f'xgb_pred_{bucket}_upper'])

        # Add KNN predictions
        knn_cols.extend([f'knn_pred_{bucket}', f'knn_pred_{bucket}_lower', f'knn_pred_{bucket}_upper'])

    # Fill missing columns with appropriate values
    for col in set(xgb_cols + knn_cols):
        if col not in predictions_df.columns:
            if 'actual_' in col:
                predictions_df[col] = np.nan
            elif '_lower' in col or '_upper' in col:
                predictions_df[col] = np.nan
            elif col == 'prediction_type':
                predictions_df[col] = 'future'  # Default for predictions without this column

    # Save XGBoost predictions
    logger.info(f"Preparing XGBoost predictions for upload...")
    xgb_output = predictions_df[xgb_cols].copy()

    # Log XGBoost prediction statistics
    logger.info(f"XGBoost prediction statistics:")
    for bucket in ['y_single', 'y_small', 'y_medium', 'y_large']:
        pred_col = f'xgb_pred_{bucket}'
        if pred_col in xgb_output.columns:
            mean_pred = xgb_output[pred_col].mean()
            max_pred = xgb_output[pred_col].max()
            min_pred = xgb_output[pred_col].min()
            logger.info(f"  - {bucket}: Mean={mean_pred:.2f}, Min={min_pred:.2f}, Max={max_pred:.2f}")

    logger.info(f"Uploading {len(xgb_output):,} rows to XGBoost table...")
    start_time = datetime.now()

    xgb_output.to_gbq(
        xgb_table,
        project_id=PROJECT_ID,
        if_exists='replace'
    )

    upload_time = (datetime.now() - start_time).total_seconds()
    logger.info(f"XGBoost predictions saved successfully in {upload_time:.1f} seconds")
    logger.info(f"Upload rate: {len(xgb_output)/upload_time:.0f} rows/second")

    # Save KNN predictions
    logger.info(f"Preparing KNN predictions for upload...")
    knn_output = predictions_df[knn_cols].copy()

    # Log KNN prediction statistics
    logger.info(f"KNN prediction statistics:")
    for bucket in ['y_single', 'y_small', 'y_medium', 'y_large']:
        pred_col = f'knn_pred_{bucket}'
        if pred_col in knn_output.columns:
            mean_pred = knn_output[pred_col].mean()
            max_pred = knn_output[pred_col].max()
            min_pred = knn_output[pred_col].min()
            logger.info(f"  - {bucket}: Mean={mean_pred:.2f}, Min={min_pred:.2f}, Max={max_pred:.2f}")

    logger.info(f"Uploading {len(knn_output):,} rows to KNN table...")
    start_time = datetime.now()

    knn_output.to_gbq(
        knn_table,
        project_id=PROJECT_ID,
        if_exists='replace'
    )

    upload_time = (datetime.now() - start_time).total_seconds()
    logger.info(f"KNN predictions saved successfully in {upload_time:.1f} seconds")
    logger.info(f"Upload rate: {len(knn_output)/upload_time:.0f} rows/second")

    # Save metrics (unchanged)
    logger.info(f"Preparing metrics for {len(all_metrics)} zones...")
    metrics_rows = []

    # Track best performing zones
    zone_performance = {}

    for zone_id, zone_metrics in all_metrics.items():
        zone_avg_smape = {'xgb': 0, 'knn': 0}

        for model_type in ['xgb', 'knn']:
            for target, metrics in zone_metrics[model_type].items():
                metrics_rows.append({
                    'taxi_type': taxi_type,
                    'zone_id': zone_id,
                    'model_type': model_type,
                    'target': target,
                    'mae': metrics['mae'],
                    'rmse': metrics['rmse'],
                    'smape': metrics['smape'],
                    'timestamp': datetime.now()
                })
                zone_avg_smape[model_type] += metrics['smape']

        # Calculate average SMAPE for zone
        zone_performance[zone_id] = {
            'xgb_avg_smape': zone_avg_smape['xgb'] / 4,
            'knn_avg_smape': zone_avg_smape['knn'] / 4
        }

    metrics_df = pd.DataFrame(metrics_rows)

    logger.info(f"Metrics summary:")
    logger.info(f"  - Total metric rows: {len(metrics_df)}")
    logger.info(f"  - Unique zones: {metrics_df['zone_id'].nunique()}")
    logger.info(f"  - Model types: {metrics_df['model_type'].unique()}")
    logger.info(f"  - Targets: {metrics_df['target'].unique()}")

    # Log overall performance statistics
    for model_type in ['xgb', 'knn']:
        model_metrics = metrics_df[metrics_df['model_type'] == model_type]
        logger.info(f"\n{model_type.upper()} overall performance:")
        logger.info(f"  - Mean MAE: {model_metrics['mae'].mean():.2f}")
        logger.info(f"  - Mean RMSE: {model_metrics['rmse'].mean():.2f}")
        logger.info(f"  - Mean SMAPE: {model_metrics['smape'].mean():.1f}%")

    logger.info(f"\nUploading {len(metrics_df)} metric rows...")
    start_time = datetime.now()

    metrics_df.to_gbq(
        metrics_table,
        project_id=PROJECT_ID,
        if_exists='replace'
    )

    upload_time = (datetime.now() - start_time).total_seconds()
    logger.info(f"Metrics saved successfully in {upload_time:.1f} seconds")

    # Log top performing zones
    sorted_zones = sorted(zone_performance.items(),
                         key=lambda x: min(x[1]['xgb_avg_smape'], x[1]['knn_avg_smape']))

    logger.info(f"\nTop 3 best performing zones:")
    for zone_id, perf in sorted_zones[:3]:
        logger.info(f"  - Zone {zone_id}: XGB SMAPE={perf['xgb_avg_smape']:.1f}%, KNN SMAPE={perf['knn_avg_smape']:.1f}%")

    # Data quality check
    logger.info(f"\nData quality check:")
    logger.info(f"  - Predictions with NaN values: {predictions_df.isna().sum().sum()}")

    # Check only prediction columns for negative values
    pred_cols = [col for col in predictions_df.columns if 'pred_' in col]
    if pred_cols:
        negative_count = (predictions_df[pred_cols] < 0).sum().sum()
        logger.info(f"  - Negative predictions: {negative_count}")
    else:
        logger.info(f"  - No prediction columns found")

    logger.info(f"  - Zones with complete predictions: {len(all_predictions)}/{len(all_metrics)}")

    logger.info(f"="*80)
    logger.info(f"All results saved successfully for {taxi_type} taxi")
    logger.info(f"="*80)

In [ ]:
def save_models_to_gcs(zone_models, taxi_type):
    """Save trained models to GCS."""
    bucket = storage_client.get_bucket(BUCKET_NAME)

    for zone_id, models in tqdm(zone_models.items()):
        # Save XGBoost model
        xgb_path = f"{MODEL_FOLDER}/{taxi_type}/zone_{zone_id}/xgb_model.joblib"
        print(f"saveing {xgb_path}")
        blob = bucket.blob(xgb_path)
        with blob.open('wb') as f:
            joblib.dump(models['xgb'], f)

        # Save KNN model
        knn_path = f"{MODEL_FOLDER}/{taxi_type}/zone_{zone_id}/knn_model.joblib"
        print(f"saveing {knn_path}")
        blob = bucket.blob(knn_path)
        with blob.open('wb') as f:
            joblib.dump(models['knn'], f)

        # Save preprocessor
        preprocessor_path = f"{MODEL_FOLDER}/{taxi_type}/zone_{zone_id}/preprocessor.joblib"
        blob = bucket.blob(preprocessor_path)
        with blob.open('wb') as f:
            joblib.dump(models['preprocessor'], f)

    logger.info(f"Saved {len(zone_models)} zone models to GCS")

In [ ]:
def main():
    """Main pipeline execution."""
    logger.info("Starting NYC Taxi Zone-Based Passenger Predictor Pipeline")

    for taxi_type in tqdm(TAXI_TYPES):
        logger.info(f"\nProcessing {taxi_type} taxi...")

        try:
            # Extract features
            df = extract_features_from_bigquery(taxi_type)
            df = prepare_features(df)

            # Train models for all zones
            zone_models, all_metrics, residual_stds, last_date, test_predictions  = train_all_zones(df, taxi_type)

            # Generate predictions for each zone
            all_predictions = []
            for zone_id in tqdm(zone_models.keys(), desc="Generating predictions"):
                zone_data = df[df['zone_id'] == zone_id]
                zone_pred = predict_future_iterative(
                    zone_data, zone_models, zone_id, last_date, residual_stds, FORECAST_DAYS
                )
                if zone_pred is not None:
                    all_predictions.append(zone_pred)

            # Save results
            save_results(all_predictions, all_metrics, test_predictions, taxi_type)
            save_models_to_gcs(zone_models, taxi_type)

            logger.info(f"Completed processing for {taxi_type} taxi")
            print(f"Completed processing for {taxi_type} taxi")

        except Exception as e:
            logger.error(f"Error processing {taxi_type}: {str(e)}")
            continue

    logger.info("\nPipeline completed successfully!")

    # Create summary views with confidence intervals
    for model_type in ['xgb', 'knn']:
        summary_query = f"""
        CREATE OR REPLACE VIEW `{PROJECT_ID}.{OUTPUT_DATASET}.fleet_recommender_summary_new_{model_type}` AS
        WITH combined AS (
            SELECT * FROM `{PROJECT_ID}.{OUTPUT_DATASET}.fleet_recommender_yellow_predictions_new_{model_type}`
            UNION ALL
            SELECT * FROM `{PROJECT_ID}.{OUTPUT_DATASET}.fleet_recommender_green_predictions_new_{model_type}`
        )
        SELECT
            taxi_type,
            date,
            SUM({model_type}_pred_y_single) as total_single_passengers,
            SUM({model_type}_pred_y_small) as total_small_groups,
            SUM({model_type}_pred_y_medium) as total_medium_groups,
            SUM({model_type}_pred_y_large) as total_large_groups,
            SUM({model_type}_pred_y_single + {model_type}_pred_y_small +
                {model_type}_pred_y_medium + {model_type}_pred_y_large) as total_expected_trips,
            -- Confidence intervals
            SUM({model_type}_pred_y_single_lower + {model_type}_pred_y_small_lower +
                {model_type}_pred_y_medium_lower + {model_type}_pred_y_large_lower) as total_expected_trips_lower,
            SUM({model_type}_pred_y_single_upper + {model_type}_pred_y_small_upper +
                {model_type}_pred_y_medium_upper + {model_type}_pred_y_large_upper) as total_expected_trips_upper,
            COUNT(DISTINCT zone_id) as active_zones
        FROM combined
        GROUP BY taxi_type, date
        ORDER BY taxi_type, date
        """

        bq_client.query(summary_query).result()
        logger.info(f"Created {model_type} summary view with confidence intervals")


In [ ]:
main()